In [1]:
import numpy as np
np.set_printoptions(suppress=True)

from dm_control import mujoco
import matplotlib.pylab as plt

import time

import pinocchio as pin

import robot_properties_solo
from robot_properties_solo.config import Solo12Config

from mujoco_utils import MujocoWrapper

In [2]:
# Load the model of Solo12 and create a mujoco simulation.
physics = mujoco.Physics.from_xml_path('world_solo12.xml')

# Load Solo12 as a pinocchio robot.
pin_robot = Solo12Config.buildRobotWrapper()

# Create wrapper to make interaction between Mujoco <-> Pinocchio easy.
robot = MujocoWrapper(physics, pin_robot, ['base'], [5, 9, 13, 17], False)

In [3]:
from mim_control_cpp import (
    ImpedanceController,
    CentroidalPDController,
    CentroidalForceQPController,
)

robot_config = Solo12Config
robot.nb_ee = 4 # HACK
mu = 0.6
kc = [200, 200, 200]
dc = [50, 50, 50]
kb = [100, 100, 200]
db = [50.0, 50.0, 200.0]
qp_penalty_weights = [5e5, 5e5, 5e5, 1e6, 1e6, 1e6]

# Create impedance controllers
root_name = "universe"
endeff_names = Solo12Config.end_effector_names
end_eff_ids = [
    pin_robot.model.getFrameId(ee_name) for ee_name in endeff_names
]
imp_ctrls = [ImpedanceController() for _ in endeff_names]
for i, c in enumerate(imp_ctrls):
    c.initialize(pin_robot.model, root_name, endeff_names[i])

# create centroidal controller
q_init = np.zeros(pin_robot.nq)
q_init[7] = 1
centrl_pd_ctrl = CentroidalPDController()
centrl_pd_ctrl.initialize(
    2.5, np.diag(pin_robot.mass(q_init)[3:6, 3:6])
)
force_qp = CentroidalForceQPController()
force_qp.initialize(robot.nb_ee, mu, qp_penalty_weights)

# Reset the robot to some initial state.
q0 = robot_config.initial_configuration
q0[0] = 0.0
dq0 = robot_config.initial_velocity
robot.reset_state(q0, dq0)

# Desired center of mass position and velocity.
x_com = [0.0, 0.0, 0.25]
xd_com = [0.0, 0.0, 0.0]
# The base should be flat.
x_ori = [0.0, 0.0, 0.0, 1.0]
x_angvel = [0.0, 0.0, 0.0]
# Alle end-effectors are in contact.
cnt_array = robot.nb_ee * [1]

# impedance gains
kp = np.array([200, 200, 200, 0, 0, 0])
kd = np.array([10.0, 10.0, 10.0, 0, 0, 0])

# Desired leg length
x_des = [
    0.195,
    0.147,
    0.015,
    0.195,
    -0.147,
    0.015,
    -0.195,
    0.147,
    0.015,
    -0.195,
    -0.147,
    0.015,
]
xd_des = pin.Motion(np.zeros(3), np.zeros(3))

In [4]:
from dm_control import _render
from dm_control.viewer import gui
from dm_control.viewer import renderer
from dm_control.viewer import viewer
from dm_control.viewer import views

_MAX_FRONTBUFFER_SIZE = 2048
_MAX_FRONTBUFFER_SIZE = 2048

width = 1024
height=768
title = 'Mujoco Simulator'

viewport = renderer.Viewport(width, height)
window = gui.RenderWindow(width, height, title)

viewer_layout = views.ViewportLayout()
viewer = viewer.Viewer(
    viewport, window.mouse, window.keyboard)

if 'render_surface' in globals():
    render_surface.free()
if 'renderer_' in globals():
    renderer_.release()
    
render_surface = _render.Renderer(
    max_width=_MAX_FRONTBUFFER_SIZE, max_height=_MAX_FRONTBUFFER_SIZE)
renderer_ = renderer.OffScreenRenderer(
    physics.model, render_surface)
renderer_.components += viewer_layout
viewer.initialize(physics, renderer_, touchpad=True)

last_tick = time.time()

def tick():
    global last_tick
    
    elapsed = time.time() - last_tick
    last_tick = time.time()
    
    if elapsed > 0:
        with viewer.perturbation.apply(False):
            while elapsed > 0:
                # Run the controller.
#                 q, dq = robot.get_state()
# #                 jnt_des = np.array([0., 0.8, -1.6, 0., 0.8, -1.6, 0., -0.8, 1.6, 0., -0.8, 1.6])
#                 jnt_des = np.zeros((12))
#                 tau = 5. * (jnt_des - q[7:]) - 0.05 * dq[6:]    
#                 robot.send_joint_command(tau)

                # Read the final state and forces after the stepping.
                q, dq = robot.get_state()

                quat = pin.Quaternion(q[6], q[3], q[4], q[5])
                quat.normalize()

                # computing forces to be applied in the centroidal space
                centrl_pd_ctrl.run(
                    kc,
                    dc,
                    kb,
                    db,
                    q[:3],
                    x_com,
                    quat.toRotationMatrix().dot(dq[:3]), # local to world frame
                    xd_com,
                    q[3:7],
                    x_ori,
                    dq[3:6],
                    x_angvel,
                )

                w_com = np.zeros(6)
                w_com[2] += 9.81 * 2.5
                w_com += centrl_pd_ctrl.get_wrench()

                # distributing forces to the active end effectors
                pin_robot.framesForwardKinematics(q)
                com = pin_robot.com(q)
                rel_eff = np.array(
                    [pin_robot.data.oMf[i].translation - com for i in end_eff_ids]
                ).reshape(-1)
                force_qp.run(w_com, rel_eff, cnt_array)
                ee_forces = force_qp.get_forces()

                # passing forces to the impedance controller
                tau = np.zeros(robot.nb_dof)
                for i, imp_ctrl in enumerate(imp_ctrls):
                    imp_ctrl.run(
                        q,
                        dq,
                        kp,
                        kd,
                        1.0,
                        pin.SE3(np.eye(3), np.array(x_des[3 * i : 3 * (i + 1)])),
                        pin.Motion(xd_des),
                        pin.Force(
                            np.array(ee_forces[3 * i : 3 * (i + 1)]), np.zeros(3)
                        ),
                    )
                    tau += imp_ctrl.get_joint_torques()

                # passing torques to the robot
                robot.send_joint_command(tau)

                # Run the physics
                physics.step()

                elapsed -= 0.001
    
    # Update the viewport and perform the rendering.
    viewport.set_size(*window.shape)
    viewer.render()
    
    return renderer_.pixels

window.event_loop(tick_func=tick)
window.close()
del window